In [2]:
import pandas as pd

In [3]:
user_data = pd.read_csv('data/ira_users_csv_hashed.csv')

In [6]:
user_data.head()

,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,account_language
0,94c383089f0dd9993020276bd01113ecb5935ad860bfa6...,94c383089f0dd9993020276bd01113ecb5935ad860bfa6...,94c383089f0dd9993020276bd01113ecb5935ad860bfa6...,USA,Conservative. Politics. Pro-Life. Writer. #Go...,NaN,1541,1676,2014-06-13,en
1,799bb58d3c064d3884fdc4604c74068ff059b60c0b3464...,799bb58d3c064d3884fdc4604c74068ff059b60c0b3464...,799bb58d3c064d3884fdc4604c74068ff059b60c0b3464...,Раша,На воре и шапка горит,NaN,299,324,2013-12-22,en
2,0d33d19fbadccaad0033a0a0d6ec7f9919e21da60ea958...,0d33d19fbadccaad0033a0a0d6ec7f9919e21da60ea958...,0d33d19fbadccaad0033a0a0d6ec7f9919e21da60ea958...,"Чеченская республика, Россия",Я никогда не хотел стать президентом. У меня в...,NaN,4094,0,2016-11-25,ru
3,efbe6ab452cf84134d5e7764cc1ce176067c8224fd8868...,efbe6ab452cf84134d5e7764cc1ce176067c8224fd8868...,efbe6ab452cf84134d5e7764cc1ce176067c8224fd8868...,US,❥ ❣ my sweethearts)),NaN,140,191,2013-08-11,en
4,d1e5ee3856d4af872b7bb31638fb86fa0a8ee3287d8ca8...,d1e5ee3856d4af872b7bb31638fb86fa0a8ee3287d8ca8...,d1e5ee3856d4af872b7bb31638fb86fa0a8ee3287d8ca8...,NaN,NaN,NaN,119,1152,2016-07-06,de


In [23]:
tweets = pd.read_csv('data/ira_tweets_csv_hashed.csv')
tweets = tweets.iloc[0:100]
tweets.head()

/Users/kay/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices
0,877919995476496385,249064136b1c5cb00a705316ab73dd9b53785748ab757f...,249064136b1c5cb00a705316ab73dd9b53785748ab757f...,249064136b1c5cb00a705316ab73dd9b53785748ab757f...,"Москва, Россия",Я примерный семьянин!,NaN,132,120,2013-12-07,...,NaN,NaN,0.0,0.0,0.0,0.0,[],[http://ru-open.livejournal.com/374284.html],[2572896396],NaN
1,492388766930444288,0974d5dbee4ca9bd6c3b46d62a5cbdbd5c0d86e196b624...,0974d5dbee4ca9bd6c3b46d62a5cbdbd5c0d86e196b624...,0974d5dbee4ca9bd6c3b46d62a5cbdbd5c0d86e196b624...,Россия,Телефонист .Изучение истории Игра в любитель...,NaN,74,8,2014-03-15,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,[http://pyypilg33.livejournal.com/11069.html],NaN,NaN
2,719455077589721089,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,Рязань,волны так и плещут фиолетовой волной,NaN,165,454,2014-04-29,...,NaN,NaN,0.0,0.0,0.0,0.0,[],[https://www.youtube.com/watch?v=9GvpImWxTJc],[40807205],NaN
3,536179342423105537,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,Рязань,волны так и плещут фиолетовой волной,NaN,165,454,2014-04-29,...,NaN,NaN,0.0,0.0,0.0,0.0,[STOPNazi],NaN,NaN,NaN
4,841410788409630720,a53ed619f1dea6015c7c878bf744b0eefe8f7272dccf34...,a53ed619f1dea6015c7c878bf744b0eefe8f7272dccf34...,a53ed619f1dea6015c7c878bf744b0eefe8f7272dccf34...,NaN,Отвечаю на любой #ВопросПрезиденту,http://t.co/3CVqbMQFbf,4430,4413,2012-02-25,...,NaN,NaN,0.0,0.0,3.0,4.0,[],[https://goo.gl/fBp94X],NaN,NaN


In [25]:
len(tweets)

100

In [38]:
from urllib.parse import urlparse
import requests

for i, row in tweets.iterrows():
    if not isinstance(row['urls'], str):
        tweets.drop(i)
    elif row['urls'][1: -1] == '':
        tweets.drop(i)
    else:
        url = row['urls'][1: -1]
        res = requests.head(url)
        hostname = urlparse(res.headers.get('location')).hostname
        if hostname:
            print(i, url, '>', hostname)
        else:
            tweets.drop(i)
        
#     print(row)
#     if isinstance(urls, str):
#         url = urls[1: -1]
#         if url:
# #             hostname = urllib.parse.urlparse(url).hostname
#             res = requests.head(url)
#             hostname = urlparse(res.headers.get('location')).hostname
#             if hostname:
#                 print(hostname, type(hostname))



0 http://ru-open.livejournal.com/374284.html > ru-open.livejournal.com
1 http://pyypilg33.livejournal.com/11069.html > pyypilg33.livejournal.com
4 https://goo.gl/fBp94X > www.putin-today.ru
5 https://goo.gl/9w5hso > www.putin-today.ru
7 http://bit.ly/1Rizso9 > sibkray.ru
9 http://vesti.ru/t?2686779 > www.vesti.ru
13 http://bit.ly/1PMZqPI > progorod43.ru
14 http://bit.ly/1qYzyIL > progorod43.ru
15 http://bit.ly/1TK58o0 > progorod43.ru
16 http://bit.ly/22rNBnQ > progorod43.ru
17 http://bit.ly/1h8fA98 > progorod43.ru
18 http://bit.ly/1pjYCsY > progorod43.ru
19 https://goo.gl/cxchK2 > www.gtrk-vyatka.ru
20 https://goo.gl/3W1BDT > www.gtrk-vyatka.ru
21 http://bit.ly/1ZW60eL > progorod43.ru
22 http://dlvr.it/Nb8xqd > gorodkirov.ru
23 http://dlvr.it/NxBKlR > www.gtrk-vyatka.ru
24 http://dlvr.it/PjZQ1z > www.gtrk-vyatka.ru
25 https://goo.gl/MplXgE > www.gtrk-vyatka.ru
26 https://goo.gl/vC26US > www.gtrk-vyatka.ru
27 http://dlvr.it/Pb4w8b > www.gtrk-vyatka.ru
28 http://dlvr.it/Ppg179 > gorodkir